<a href="https://colab.research.google.com/github/Storm00212/JARVIS/blob/main/JARVIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain --quiet


In [3]:
!pip install sentence-transformers --quiet

In [4]:
!pip install chromadb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 112.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.5 MB/s 

In [5]:
!pip install pypdf --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 11.7 MB/s eta 0:00:00


In [6]:
!pip install transformers --quiet

In [7]:
!pip install accelerate --quiet

In [8]:
!pip install  bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.9 MB/s eta 0:00:00


In [9]:
!pip install torch torchvision --quiet

In [10]:
from pypdf import PdfReader

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

# Upload a PDF
from google.colab import files
uploaded = files.upload()

pdf_path = list(uploaded.keys())[0]
raw_text = extract_text_from_pdf(pdf_path)
raw_text[:500]


Saving A textbook of Electrical Technology B. L. Thereja All Volumes ( PDFDrive.pdf to A textbook of Electrical Technology B. L. Thereja All Volumes ( PDFDrive.pdf


'FIRST MULTICOLOUR EDITION\nA TEXTBOOK OF\nELECTRICAL\nTECHNOLOG Y\nALL VOLUME\nBASIC ELECTRICAL ENGINEERING\nIN\nS.I. SYSTEM OF UNITS\n\nA TEXTBOOK OF\nELECTRICAL\nTECHNOLOGY\nVOLUME I\nBASIC ELECTRICAL ENGINEERING\nIN\nS.I. SYSTEM OF UNITS\n(Including rationalized M.K.S.A. System)\nFor the Examinations of B.E. (Common Course to All Branches), B.Tech.,\nB.Sc. (Engg), Sec. A & B of AMIE(I), A.M.I.E.E. (London), I.E.R.E. (London),\nGrade I.E.T.E., Diploma and other Competitive Examinations\nB.L. THERAJA\nA.K. THERAJA\n'

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=80,
)

chunks = splitter.split_text(raw_text)
len(chunks)


8708

In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Updated: Use chromadb.PersistentClient for persistent storage
client = chromadb.PersistentClient(path="db")

# Ensure the collection doesn't already exist from a previous run to avoid errors
try:
    collection = client.get_or_create_collection("jarvis_docs")
except Exception as e:
    print(f"Error getting or creating collection: {e}")
    # If you want to delete and recreate the collection, uncomment the following lines:
    # client.delete_collection(name="jarvis_docs")
    # collection = client.create_collection("jarvis_docs")

# Clear existing data in the collection if any to avoid ID conflicts if running multiple times
if collection.count() > 0:
    print("Clearing existing documents in the collection...")
    collection.delete(ids=collection.get()['ids'])

for i, chunk in enumerate(chunks):
    emb = embedder.encode(chunk).tolist()
    collection.add(documents=[chunk], embeddings=[emb], ids=[str(i)])

# client.persist() is no longer needed with PersistentClient as it persists automatically
print(f"Successfully added {collection.count()} documents to the collection.")